In [9]:
import pandas as pd
import re

# Drop

In [30]:
# DROP datasets
df_8b = pd.read_csv('/Users/log/Downloads/fs_inst_llama_sambanova_8b_temp_10_full.csv')
df_8b['model'] = '8b'
df_70b = pd.read_csv('/Users/log/Downloads/fs_inst_llama_sambanova_70b_temp_10_longest (5).csv')
df_70b['model'] = '70b'
df_405B = pd.read_csv('/Users/log/Downloads/fs_inst_llama_sambanova_405b_temp_10_longest (6).csv')
df_405B['model'] = '405B'
df_flash = pd.read_csv('/Users/log/Downloads/fs_inst_gemini-1.5-flash-002_temp_10_longest (4).csv')
df_flash['model'] = 'flash'
df_pro = pd.read_csv('/Users/log/Downloads/fs_inst_gemini-1.5-pro-002_temp_10_longest (2).csv')
df_pro['model'] = 'pro'

drop_responses = pd.concat([df_8b, df_70b, df_405B, df_flash, df_pro])

# models = df_combined['model'].unique().tolist()  # Explicit order if needed

# selected_ids = set()
# result_dfs = []

# for model in models:
#     # Filter current model's entries not already selected
#     available = df_combined[(df_combined['model'] == model) & (~df_combined['id'].isin(selected_ids))]
    
#     # Sample 100 unique IDs
#     sampled = available.drop_duplicates(subset='id').sample(n=50, random_state=42)  # Adjust random_state if needed
    
#     if len(sampled) < 50:
#         raise ValueError(f"Model {model} lacks 100 unique IDs after removing overlaps.")
    
#     # Update selected IDs and store results
#     selected_ids.update(sampled['id'].tolist())
#     result_dfs.append(sampled)

# # Combine results and verify
# drop_responses = pd.concat(result_dfs, ignore_index=True)
# assert drop_responses['id'].nunique() == 250, "Duplicate IDs detected in final selection."

# drop_df['id'].value_counts()
# drop_df.columns()
drop_responses
drop_responses['model'].value_counts()


model
8b       720
70b      720
405B     720
flash    720
pro      720
Name: count, dtype: int64

In [38]:
jsonl_path = '/Users/log/Github/textual_grounding/data/drop_break/test.jsonl'
data = pd.read_json(jsonl_path, lines=True)
df_drop = pd.DataFrame(columns=['id', 'question', 'model', 'answer', 'gt_number', 'isTrue'])


def extract_answer(text):
    pattern = re.compile(r"\{([-]?[0-9,]*\.?[0-9]+)\}")
    match = pattern.search(str(text))
    num_str = match.group(1) if match else "-99999999999999"
    num_str_clean = num_str.replace(',', '')  # '9759417'
    numeric_value = float(num_str_clean)      # 9759417.0 (float)
    return numeric_value

for idx, row in drop_responses.iterrows():
    response_text = row['answer']
    
    # Compare to ground-truth
    gt_row = data.loc[data['id'] == row['id']]
    gt_answer = gt_row['answer'].iloc[0]  # or answer[0], etc.
    
    if len(gt_answer) > 1:
        continue
    extracted_answer = extract_answer(response_text)
    # print(f"ID: {row['id']}")
    # print(f"Model Answer Text: {response_text}")
    # print(f"Extracted numeric answer: {extracted_answer}")
    # gt_number = float(gt_answer[-1][0])  # 54.4
    # if extracted_answer == -99999999999999:
    #     continue
    gt1 = str(gt_answer[0][0])
    # gt2 = ""
    # if len(gt_answer) == 2:
    #     gt2 = str(gt_answer[1][0])
    # gt_number = gt1 + " " + gt2
    gt_number = gt1
    row['gt_number'] = float(gt_number)
    # mismatched_rows = pd.concat([mismatched_rows, row.to_frame().T], ignore_index=True)
    # print(f"Ground-truth answer: {gt_number}")
    # print("----")
    
    
    # Check if the extracted answer matches the ground-truth
    # print(extracted_answer, gt_number)
    # if extracted_answer == gt_number:
    strip_ans = row['answer'].replace(',', '').replace('.', '')
    # if abs(float(extracted_answer) - float(gt_number)) <= 0.00001:
    # print(str(int(row['gt_number'])))
    row['extract_answer'] = extracted_answer
    row['gt_number'] = gt_number
    if str(float(row['gt_number'])) not in response_text[-10::] and abs(float(extracted_answer) - float(gt_number)) > 0.1:
        # if extracted_answer == -99999999999999 and row['id'] not in good_questions:
        # print(row['answer'])
        # print(gt_number)
        # print()
            # continue
        # if abs(float(extracted_answer) - float(gt_number)) <= 0.001:
        #     continue
        row['isTrue'] = False

        # if row['id'] in good_questions:
        #     print(row['id'])
        df_drop = pd.concat([df_drop, row.to_frame().T], ignore_index=True)
    # if abs(float(extracted_answer) - float(gt_number)) < 0.1: # correct answer
    else:
        # print('aoeuaoe')
        row['isTrue'] = True
        df_drop = pd.concat([df_drop, row.to_frame().T], ignore_index=True)
    #     mismatched_rows = pd.concat([mismatched_rows, row.to_frame().T], ignore_index=True)
    # if extracted_answer == gt_number: # correct
    #     row['gt_number'] = gt_number
    #     mismatched_rows = pd.concat([mismatched_rows, row.to_frame().T], ignore_index=True)

# print(df_drop.head())
print(df_drop['isTrue'].value_counts())
# print(df_drop['model'].value_counts())

for model, group in df_drop.groupby('model'):
    print(f"Model: {model}")
    print(group['isTrue'].value_counts())
    print("\n")  # Add a newline for better readability


isTrue
True     2077
False     693
Name: count, dtype: int64
Model: 405B
isTrue
True     491
False     63
Name: count, dtype: int64


Model: 70b
isTrue
True     427
False    127
Name: count, dtype: int64


Model: 8b
isTrue
True     289
False    265
Name: count, dtype: int64


Model: flash
isTrue
True     388
False    166
Name: count, dtype: int64


Model: pro
isTrue
True     482
False     72
Name: count, dtype: int64




need 250 drop responses

50 responses per model
25 true 25 false

In [40]:
import pandas as pd

# Initialize an empty DataFrame to store the final results
final_df = pd.DataFrame(columns=['id', 'question', 'model', 'answer', 'gt_number', 'isTrue'])

# Track used IDs to ensure uniqueness across all models
used_ids = set()

# Iterate over each model
for model in df_drop['model'].unique():
    # Filter the DataFrame for the current model
    model_df = df_drop[df_drop['model'] == model]
    
    # Separate true and false responses
    true_responses = model_df[model_df['isTrue'] == True]
    false_responses = model_df[model_df['isTrue'] == False]
    
    # Filter out IDs that have already been used
    true_responses = true_responses[~true_responses['id'].isin(used_ids)]
    false_responses = false_responses[~false_responses['id'].isin(used_ids)]
    
    # Sample 25 true and 25 false responses, ensuring unique IDs
    sampled_true = true_responses.drop_duplicates(subset=['id']).sample(n=25, random_state=1)
    sampled_false = false_responses.drop_duplicates(subset=['id']).sample(n=25, random_state=1)
    
    # Add the sampled IDs to the used_ids set
    used_ids.update(sampled_true['id'].tolist())
    used_ids.update(sampled_false['id'].tolist())
    
    # Combine the sampled responses
    sampled_responses = pd.concat([sampled_true, sampled_false])
    
    # Append to the final DataFrame
    final_df = pd.concat([final_df, sampled_responses])

# Reset the index of the final DataFrame
final_df.reset_index(drop=True, inplace=True)

# Check the final DataFrame
print(final_df['isTrue'].value_counts())
print(final_df['model'].value_counts())

# Verify that all IDs are unique
assert final_df['id'].nunique() == final_df.shape[0], "There are duplicate IDs in the final DataFrame."

# Save the final DataFrame to a CSV file if needed
final_df.to_csv('sampled_responses.csv', index=False)

isTrue
True     125
False    125
Name: count, dtype: int64
model
8b       50
70b      50
405B     50
flash    50
pro      50
Name: count, dtype: int64


In [41]:
for model, group in final_df.groupby('model'):
    print(f"Model: {model}")
    print(group['isTrue'].value_counts())
    print("\n")  # Add a newline for better readability

Model: 405B
isTrue
True     25
False    25
Name: count, dtype: int64


Model: 70b
isTrue
True     25
False    25
Name: count, dtype: int64


Model: 8b
isTrue
True     25
False    25
Name: count, dtype: int64


Model: flash
isTrue
True     25
False    25
Name: count, dtype: int64


Model: pro
isTrue
True     25
False    25
Name: count, dtype: int64




# Symbolic

In [ ]:
# main split
df_8b_incorrect = pd.read_csv('/Users/log/Github/textual_grounding/logan/SYMBOLIC_data/8b_main_incorrect_responses.csv')
df_8b_incorrect['model'] = '8b'
df_8b_correct = pd.read_csv('/Users/log/Github/textual_grounding/logan/SYMBOLIC_data/8b_main_correct_responses.csv')
df_8b_correct['model'] = '8b'

df_flash_incorrect = pd.read_csv('/Users/log/Github/textual_grounding/logan/SYMBOLIC_data/gflash_main_incorrect_responses.csv')
df_flash_incorrect['model'] = 'flash'
df_flash_correct = pd.read_csv('/Users/log/Github/textual_grounding/logan/SYMBOLIC_data/gflash_main_correct_responses.csv')
df_flash_correct['model'] = 'flash'

df_main = pd.concat([df_8b_incorrect, df_8b_correct, df_flash_incorrect, df_flash_correct])

# p2 split
df_70b_incorrect = pd.read_csv('/Users/log/Github/textual_grounding/logan/SYMBOLIC_data/70b_p2_40_incorrect_responses.csv')
df_70b_incorrect['model'] = '70b'
df_70b_correct = pd.read_csv('/Users/log/Github/textual_grounding/logan/SYMBOLIC_data/70b_p2_40_correct_responses.csv')
df_70b_correct['model'] = '70b'


df_405B_incorrect = pd.read_csv('/Users/log/Github/textual_grounding/logan/SYMBOLIC_data/405b_p2_incorrect_responses.csv')
df_405B_incorrect['model'] = '405B'
df_405B_correct = pd.read_csv('/Users/log/Github/textual_grounding/logan/SYMBOLIC_data/405b_p2_correct_responses.csv')
df_405B_correct['model'] = '405B'

df_pro = pd.read_csv('/Users/log/Github/textual_grounding/logan/SYMBOLIC_data/gemini_pro_p2_subset.csv')
df_pro['model'] = 'pro'

df_p2 = pd.concat([df_70b_incorrect, df_70b_correct, df_405B_incorrect, df_405B_correct])





